In [1]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


In [2]:
os.environ['GROQ_API_KEY'] = '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'

In [3]:
llm = ChatGroq(temperature=0, groq_api_key=os.getenv("GROQ_API_KEY"), model_name="llama-3.1-70b-versatile")

In [13]:
prompt_extract = PromptTemplate.from_template(
            """
            ###  your are interactive chatbot and answer the question 
            ### great and answer the question 
            """
        )
chain_extract = llm
# res = chain_extract.invoke(input={'hi hello could you give me this item for 100 rupe'})


In [5]:

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = chain_extract.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
config = {"configurable": {"thread_id": "abc123"}}

In [20]:

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = llm.invoke(input_messages, config)
print(output.content)  # output contains all messages in state

Hi Bob, it's nice to meet you. Is there something I can help you with or would you like to chat?


In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | llm
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [21]:
query = "Hi ! what is my name."

input_messages = [HumanMessage(query)]
output = llm.invoke(input_messages, config)
print(output.content)  # output contains all messages in state

I don't have any information about your name. This is the beginning of our conversation, and I don't have any prior knowledge about you. If you'd like to share your name, I'd be happy to chat with you.


In [1]:
import pandas as pd

# List of items with original and selling prices
items = [
    {"Item": "Smartphone", "Original Price": 500, "Selling Price": 550},
    {"Item": "Laptop", "Original Price": 800, "Selling Price": 880},
    {"Item": "Headphones", "Original Price": 50, "Selling Price": 60},
    {"Item": "Smartwatch", "Original Price": 150, "Selling Price": 165},
    {"Item": "Bluetooth Speaker", "Original Price": 100, "Selling Price": 120},
    {"Item": "Tablet", "Original Price": 300, "Selling Price": 350},
    {"Item": "Camera", "Original Price": 600, "Selling Price": 680},
    {"Item": "Gaming Console", "Original Price": 400, "Selling Price": 450},
    {"Item": "Wireless Mouse", "Original Price": 20, "Selling Price": 25},
    {"Item": "Keyboard", "Original Price": 30, "Selling Price": 40},
    {"Item": "Monitor", "Original Price": 200, "Selling Price": 230},
    {"Item": "Printer", "Original Price": 150, "Selling Price": 175},
    {"Item": "External Hard Drive", "Original Price": 80, "Selling Price": 95},
    {"Item": "Router", "Original Price": 70, "Selling Price": 85},
    {"Item": "Television", "Original Price": 900, "Selling Price": 1000},
    {"Item": "Air Conditioner", "Original Price": 1200, "Selling Price": 1300},
    {"Item": "Microwave Oven", "Original Price": 250, "Selling Price": 275},
    {"Item": "Refrigerator", "Original Price": 1000, "Selling Price": 1100},
    {"Item": "Washing Machine", "Original Price": 700, "Selling Price": 780},
    {"Item": "Vacuum Cleaner", "Original Price": 150, "Selling Price": 175}
]

# Create a pandas DataFrame from the list
df = pd.DataFrame(items)

# Output the DataFrame to a CSV file
df.to_csv("ecommerce_items.csv", index=False)

print("CSV file saved successfully!")


CSV file saved successfully!


In [2]:
# display_items.py

import pandas as pd
from tabulate import tabulate

# Read the CSV file
df = pd.read_csv("ecommerce_items.csv")

# Select only 'Item' and 'Selling Price' columns
data = df[['Item', 'Selling Price']][10:]

# Display the data in a nice tabular format
print(tabulate(data, headers='keys', tablefmt='grid', showindex=False))



+---------------------+-----------------+
| Item                |   Selling Price |
+=====================+=================+
| Smartphone          |             550 |
+---------------------+-----------------+
| Laptop              |             880 |
+---------------------+-----------------+
| Headphones          |              60 |
+---------------------+-----------------+
| Smartwatch          |             165 |
+---------------------+-----------------+
| Bluetooth Speaker   |             120 |
+---------------------+-----------------+
| Tablet              |             350 |
+---------------------+-----------------+
| Camera              |             680 |
+---------------------+-----------------+
| Gaming Console      |             450 |
+---------------------+-----------------+
| Wireless Mouse      |              25 |
+---------------------+-----------------+
| Keyboard            |              40 |
+---------------------+-----------------+
| Monitor             |           